In [1]:
from bs4 import BeautifulSoup
import csv
import re
import os 
import pandas as pd
import glob

In [2]:
data = []

# reading html files from folder

#'Riv_Shelter_HTML_Files'is the name for the shelter HTML Files Riverside
#'Riv_Food_Pantry_HTML_Files'is the name for the food pantry HTML Files for Riverside 
#'Riv_Mental_Health_HTML_Files'is the name for the mental health HTML Files for Riverside
#'Downtown_LA_Food_Pantry_HTML_Files' is the name for the food pantry HTML Files for downtown LA
#'Irvine_Mental_Health_HTML_Files' is the name for the mental health HTML Files for Irvine
folder_path = 'Chenzi Food Pantry HTML'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')


        # Find the <ul> element with class "best-programs"
        ul_element = soup.find('ul', {'class': 'best-programs'})

        # Find all <li> elements within the <ul>
        li_elements = ul_element.find_all('li', {'class': 'search-result card card-v3 program-info'})

        # Loop through the found <li> elements and extract labels and values
        for li_element in li_elements:
            # Use BeautifulSoup to parse the HTML of each li_element
            li_soup = BeautifulSoup(str(li_element), 'html.parser')

            # Find the program heading 
            program_heading_element = li_soup.find('div', {'class': 'card-heading'})
            Service_name = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'}).text
            print('Service name: ', Service_name)
    
            # Extract program URL
            program_url_element = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'})
            program_url = program_url_element['href'] if program_url_element else None
            Service_url = "https://www.auntbertha.com/"+program_url
            print('URL: ', Service_url)
            
            is_reviewed = False

            # Reviewed on
            Reviewd_on_element = program_heading_element.find('div', {'class': 'last-reviewed'})
            if Reviewd_on_element:  # Check if the element exists
                Reviewd_on_text = Reviewd_on_element.get_text(strip=True)  # Clean up the text
                match = re.search(r'\d{2}/\d{2}/\d{4}', Reviewd_on_text)
                Reviewd_on = match.group() if match else None
                if Reviewd_on:  # If we found a review date
                    is_reviewed = True  # Set the flag to True
                print("Reviewed on:", Reviewd_on)
            else:
                Reviewd_on = None  # Set to None if not found
                print("No review date found.")
    
            # Access class="program-tags"
            program_tags = li_soup.find('div', {'class': 'program-tags'})

            # Main Services
            main_s = []
            main_service_list = program_tags.find('ul', {'class': 'list-inline'})
            main_service_items = main_service_list.find_all('li')

            for item in main_service_items:
                main_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                main_s.append(main_service)

            print("Main Services:", main_s)

            # Try to access the "Other Services" section
            other_service_list = program_tags.find('div', {'class': 'secondary-tags'})
            if other_service_list:
                other_s = []
                other_service_items = other_service_list.find('ul', {'class': 'list-inline'}).find_all('li')

                for item in other_service_items:
                    other_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                    other_s.append(other_service)
            else:
                other_s = None

            print("Other Services:", other_s)

            # Access the "Serving" section
            serving_section = program_tags.find('div', {'class': 'attribute-tags'})

            # Initialize a list to store the serving information
            serving_ = []
    
            # Find all the <li> elements within the serving section
            serving_items = serving_section.find_all('li')

            # Iterate through the serving items and extract the text from the <a> elements
            for item in serving_items:
                link = item.find('a', {'class': 'loading-on-click activity-log'})
    
                # Check if the link was found
                if link:  # Only proceed if the link exists
                    serving_text = link.text.strip()
                    serving_.append(serving_text)

            print("Serving:", serving_)

    
            # accessing next-steps-module, extract phone number, location, hours 
    
            next_steps_module = li_soup.find('div', {'class': 'next-steps-module'})

            # Extract phone number
            phone_number_elements = next_steps_module.find_all('span', {'class': 'result-next-step-item'})

            # Also find all 'a' elements with href attributes containing 'tel:'
            tel_link_elements = next_steps_module.find_all('a', href=True)

            phone_number = None  # Initialize phone number as None

            # Extract phone numbers from text-based spans
            for element in phone_number_elements:
                phone_number_text = element.text.strip() if element else None
    
                # Regex to extract digits (handling separators like spaces or hyphens)
                phone_number_matches = re.findall(r'[\d-]+', phone_number_text)
    
                # Join the digits into a single phone number string if matches are found
                if phone_number_matches:
                    phone_number = ''.join(phone_number_matches)
                    break  # Stop once we've found a phone number

            # If no phone number found from text, check the 'tel:' href links
            if not phone_number:
                for element in tel_link_elements:
                    href_value = element['href']
        
                    # Check if the href contains 'tel:' and extract digits
                    if 'tel:' in href_value:
                        phone_number_matches = re.findall(r'[\d-]+', href_value)
                        if phone_number_matches:
                            phone_number = ''.join(phone_number_matches)
                            break  # Stop once we've found a phone number

            # Print the extracted phone number
            if phone_number:
                print("Phone Number:", phone_number)
            else:
                print("No phone number found.")

            # Extract location address
            location_address_element = next_steps_module.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click map-link with-address'})
            location_address = location_address_element.text.strip() if location_address_element else None
            location_address = re.sub(r'\s+', ' ', location_address) if location_address_element else None
            print("Location Address:", location_address)

            # Extract URL
            location_url_map = location_address_element['href'] if location_address_element else None
            print("Location url map:", location_url_map)
            
            hours_info = {
                '24_hour': False,
                'Monday': None,
                'Tuesday': None,
                'Wednesday': None,
                'Thursday': None,
                'Friday': None,
                'Saturday': None,
                'Sunday': None
            }
            
            # Extract hours based on structure
            hours_element = next_steps_module.find('div', {'class': 'office-hours-schedule see-hours-dropdown'})
            if hours_element:
                # Find all the <span> elements within the hours_element
                day_spans = hours_element.find_all('span')

                # Ensure that we have even pairs of day and corresponding hours
                if len(day_spans) % 2 == 0:
                    # Iterate through the spans in pairs (day and its corresponding hours)
                    for i in range(0, len(day_spans), 2):
                        day_span = day_spans[i]          # This is the day (e.g., "Monday:")
                        hours_span = day_spans[i + 1]    # This is the corresponding hours (e.g., "Closed" or time)

                        # Extract the day name and remove the colon
                        day = day_span.text.strip()[:-1]  # Remove the ':' at the end
                        full_day = day.strip()            # Ensure there's no extra space

                        # Check if the day is valid and exists in hours_info
                        if full_day in hours_info:
                            # Check if the corresponding hours span indicates "Closed"
                            if 'Closed' in hours_span.text:
                                hours_info[full_day] = 'Closed'
                            else:
                                # Extract hours for non-closed days
                                hours_info[full_day] = hours_span.text.strip()
                else:
                    print("Unexpected hours format. Ensure that each day has corresponding hours.")

            # Handle 24-hour information if it exists
            else:
                hours_element = next_steps_module.find('span', {'class': 'result-geo-hours'})
                if hours_element:
                    # Get 24-hour text strip
                    hours_text = hours_element.get_text(strip=True)

                    # Assuming the presence of this text means 24-hour operation
                    if '24' in hours_text or '24-hour' in hours_text.lower():
                        hours_info['24_hour'] = True
                    else:
                        print("Unable to determine if the office is 24 hours.")
                else:
                    print("Hours information not available.")

            # Output the result
            print(hours_info)
    
    
            # Loop through the found <li> elements and extract labels and values
            Extra_element = li_element.find('div', {'class': 'panel-wrapper more-info-panel'})
            elig = []
            eligibility_rules_element = Extra_element.find('div', {'class': 'eligibility-rules'})
    
            # Check if eligibility_rules_element is found
            if eligibility_rules_element:
                # Check if eligibility_rules_element contains a list (ul)
                ul_element = eligibility_rules_element.find('ul')
        
                if ul_element:
                    # If it contains a list, extract list items and store them in a flat list
                    eligibility_list = [li.text.strip() for li in ul_element.find_all('li')]
                    elig.extend(eligibility_list)  # Use extend to add elements to the list directly
                    #print("Eligibility:", eligibility_list)
                else:
                    # If it doesn't contain a list, store the text as is
                    eligibility_text = eligibility_rules_element.text.strip()
                    #print("Eligibility:", [eligibility_text])  # Wrap in a list to maintain consistency
                    elig.append(eligibility_text)

            else:
                # Skip if 'eligibility-rules' class is not found
                pass
    
        
            print("Eligibility:", elig)
    
            # Extract Availability
            availability_element = Extra_element.find('strong', {'data-translate': 'Availability'})
            
            if availability_element:
                availability = availability_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                availability = "Not specified"

            # Extract Description
            description_element = Extra_element.find('strong', {'data-translate': 'Description'})
            description = description_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Languages
            languages_element = Extra_element.find('strong', {'data-translate': 'Languages'})
            languages = languages_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            languages_data = [lang.strip() for lang in languages.split(',')]
            
            # Extract Cost
            cost_element = Extra_element.find('strong', string='Cost:')
            
            if cost_element:  # Check if cost_element exists
                cost = cost_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                cost = "Not specified"  # Default value if cost_element is not found
            
            # Extract Website URLs if they exist, or set them to None
            website_element = Extra_element.find('div', {'data-translation': 'Website'})
            website_url = website_element.find_next('a', {'class': 'activity-log descriptionProgramWebsite'})['href'] if website_element else None

            # Extract Facebook and Twitter URLs if they exist, or set them to None
            facebook_element = Extra_element.find('strong', {'data-translate': 'Facebook'})
            facebook_url = facebook_element.find_next('a', {'class': 'activity-log descriptionProgramFacebook'})['href'] if facebook_element else None

            twitter_element = Extra_element.find('strong', {'data-translate': 'Twitter'})
            twitter_url = twitter_element.find_next('a', {'class': 'activity-log descriptionProgramTwitter'})['href'] if twitter_element else None

            # Extract Coverage Area
            coverage_element = Extra_element.find('strong', {'data-translate': 'Coverage Area'})
            coverage = coverage_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Initialize latitude and longitude as None by default
            latitude = None
            longitude = None
            zipcode = None

            # Find the element with class "office-hour-address"
            location_element = Extra_element.find('div', {'class': 'office-hours-address _js_address address notranslate'})

            # Check if the element exists and has the required attributes
            if location_element:
                latitude = location_element['data-latitude'] if location_element.has_attr('data-latitude') else None
                longitude = location_element['data-longitude'] if location_element.has_attr('data-longitude') else None
                print("Latitude:", latitude)
                print("Longitude:", longitude)
                
                # Extract all text within location_element
                address_text = location_element.get_text(separator=" ").strip()  # Get all text as a single string

                # Use regular expression to search for the ZIP code pattern anywhere in the text
                zip_matches = re.findall(r'(?<!\d)(\b\d{5}\b)(?!\d)', address_text)
                zipcode = int(zip_matches[-1]) if zip_matches else None
                print("ZIP Code:", zipcode)

            # Print or use the extracted values as needed
            print("Availability:", availability)
            print("Description:", description)
            print("Languages:", languages_data)
            print("Cost:", cost)
            print("Facebook URL:", facebook_url)
            print("Twitter URL:", twitter_url)
            print("Coverage Area:", coverage)

            data.append([
            Service_name,
            Service_url,
            main_s,
            other_s,
            serving_,
            phone_number,
            website_url,
            location_address,
            location_url_map,
            elig,
            availability,
            description,
            languages_data,
            cost,
            is_reviewed,
            facebook_url,
            twitter_url,
            coverage,
            latitude,
            longitude,
            zipcode,
            hours_info['24_hour'],
            hours_info['Monday'],      
            hours_info['Tuesday'],     
            hours_info['Wednesday'],   
            hours_info['Thursday'],   
            hours_info['Friday'],     
            hours_info['Saturday'],  
            hours_info['Sunday']])    
            print("***************************************************************************")

Service name:  St. Mark's Food Cupboard
URL:  https://www.auntbertha.com//saint-mark%2527s-church--philadelphia-pa--st.-mark%2527s-food-cupboard/6094953357770752?postal=19131
Reviewed on: 12/18/2024
Main Services: ['food pantry']
Other Services: ['personal care items', 'navigating the system']
Serving: ['anyone in need', 'all ages']
Phone Number: 215-735-1416101
Location Address: 1625 Locust Street, Philadelphia, PA 19103
Location url map: https://www.google.com/maps/?q=1625+Locust+Street,+Philadelphia,+PA+19103/
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': '9:30 AM - 11:00 AM  EST', 'Wednesday': '9:30 AM - 11:00 AM  EST', 'Thursday': '9:30 AM - 11:00 AM  EST', 'Friday': '9:30 AM - 11:00 AM  EST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program helps people with income at or below 185% of federal poverty guidelines.', 'This program only serves residents of the state of Pennsylvania.,']
Latitude: 39.9489257
Longitude: -75.1683325
ZIP Code: 19103
Availability: 

In [3]:
# Save to CSV file 
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv for riv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv" is the name for the food pantry csv for downtown la
csv_filename = "Philadelphia(19131-19154)_Food_Pantry_Chenzi.csv"

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # The header 
    header = [
        "Service_name",
        "Service_url",
        "Main_Services",
        "Other_Services",
        "Serving",
        "Phone_Number",
        "Website",
        "Location_Address",
        "Location_URL_Map",
        "Eligibility",
        "Availability",
        "Description",
        "Languages",
        "Cost",
        "Google_Review",
        "Facebook_URL",
        "Twitter_URL",
        "Coverage",
        "Latitude",
        "Longitude",
        "Zipcode",
        "24hour",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday"
    ]
    csv_writer.writerow(header)
    
    csv_writer.writerows(data)

In [4]:
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv"
#"FindHelp_extracted_data_downtown_la_shelter.csv"
#"FindHelp_extracted_data_downtown_la_mental_health.csv"
#"FindHelp_extracted_data_irvine_food_pantry.csv"
#"FindHelp_extracted_data_irvine_shelter.csv"
#"FindHelp_extracted_data_irvine_mental_health.csv"
df = pd.read_csv("Philadelphia(19131-19154)_Food_Pantry_Chenzi.csv")

#riv_zipcodes = [92501, 92502, 92503, 92504, 92505, 92506, 92507, 92508, 92513, 92514, 92516, 92517, 92521, 92522]

#downtown_la_zipcodes = [90012, 90013, 90014, 90015, 90017, 90021]

#irvine_zipcodes = [92602, 92603, 92604, 92606, 92612, 92614, 92617, 92618, 92620]

Philadelphia_zipcodes = [19131, 19132, 19133, 19134, 19135, 19136, 19137, 19138, 19139, 19140, 19141, 19142, 19143, 19144, 19145, 19146, 19147, 19148, 19149, 19150, 19151, 19152, 19153, 19154]

filtered_df = df[df['Zipcode'].isin(Philadelphia_zipcodes) | (df['Location_Address'].isna() & df['Longitude'].isna() & df['Latitude'].isna())]

filtered_df = filtered_df.drop_duplicates(subset = ['Website', 'Service_name'])

filtered_df.to_csv("Philadelphia(19131-19154)_Food_Pantry_Chenzi.csv", index=False, encoding='utf-8')

filtered_df

,Service_name,Service_url,Main_Services,Other_Services,Serving,Phone_Number,Website,Location_Address,Location_URL_Map,Eligibility,...,Longitude,Zipcode,24hour,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
1,Food Distribution,https://www.auntbertha.com//north10%252C-phila...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",267-908-9000,https://north10phl.org/health-wellness/,"3890 North 10th Street, Philadelphia, PA 19140",https://www.google.com/maps/?q=3890+North+10th...,['Anyone can access this program.'],...,-75.144428,19140.0,False,Closed,10:00 AM - 2:00 PM EST,Closed,10:00 AM - 2:00 PM EST,Closed,Closed,Closed
3,Food Pantry,https://www.auntbertha.com//st.-barbara%2527s-...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",215-473-1044,https://stbarbphila.org/events/food-pantry-ph,"5359 Lebanon Avenue, Philadelphia, PA 19131",https://www.google.com/maps/?q=5359+Lebanon+Av...,['Anyone can access this program.'],...,-75.231451,19131.0,False,Closed,Closed,12:00 PM - 2:00 PM EST,Closed,Closed,Closed,Closed
5,Resource Fair,https://www.auntbertha.com//the-simple-way--ph...,"['food pantry', 'clothing', 'navigating the sy...",NaN,"['anyone in need', 'all ages', 'individuals', ...",215-423-3598,https://thesimpleway.org/what-we-do/,"3234 Potter Street, Philadelphia, PA 19134",https://www.google.com/maps/?q=3234+Potter+Str...,['Anyone can access this program.'],...,-75.112190,19134.0,False,12:30 PM - 2:30 PM EST,12:30 PM - 2:30 PM EST,Closed,Closed,9:30 AM - 11:30 AM EST,Closed,Closed
6,Food Pantry,https://www.auntbertha.com//the-salvation-army...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",215-825-4601,https://easternusa.salvationarmy.org/eastern-p...,"5501 Market Street, Philadelphia, PA 19139",https://www.google.com/maps/?q=5501+Market+Str...,['Anyone can access this program.'],...,-75.231293,19139.0,False,9:00 AM - 2:00 PM EST,9:00 AM - 2:00 PM EST,9:00 AM - 2:00 PM EST,9:00 AM - 2:00 PM EST,9:00 AM - 2:00 PM EST,Closed,Closed
7,Farm to Families,https://www.auntbertha.com//st.-christopher%25...,['food pantry'],['nutrition education'],"['anyone in need', 'all ages', 'individuals', ...",484-734-0189,https://scfchildren.org/farm-to-families-initi...,"4865 Market Street, Philadelphia, PA 19139",https://www.google.com/maps/?q=4865+Market+Str...,['Must be eligible for SNAP benefits.'],...,-75.217549,19139.0,False,Closed,Closed,Closed,11:00 AM - 3:00 PM EST,Closed,Closed,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,Social Services Program,https://www.auntbertha.com//salvation-army-eas...,"['food pantry', 'meals', 'clothing', 'toys & g...",NaN,"['all ages', 'individuals', 'families', 'homel...",215-474-1010,https://easternusa.salvationarmy.org/,"5501 Market Street, Philadelphia, PA 19139",https://www.google.com/maps/?q=5501+Market+Str...,['This program serves residents of Pennsylvani...,...,-75.231293,19139.0,False,9:00 AM - 4:00 PM EST,9:00 AM - 4:00 PM EST,9:00 AM - 4:00 PM EST,9:00 AM - 4:00 PM EST,9:00 AM - 4:00 PM EST,9:00 AM - 4:00 PM EST,9:00 AM - 4:00 PM EST
114,Free Produce Markets,https://www.auntbertha.com//children%2527s-hos...,['food pantry'],NaN,"['all ages', 'individuals', 'families']",NaN,NaN,"4865 Market Street, Philadelphia, PA 19139",https://www.google.com/maps/?q=4865+Market+Str...,[],...,-75.217682,19139.0,False,Closed,Closed,Closed,9:30 AM - 10:30 AM EST,Closed,Closed,Closed
115,Food Pantry,https://www.auntbertha.com//star-of-hope-bapti...,['food pantry'],NaN,"['anyone in need', 'all ages', 'individuals', ...",NaN,https://www.starofhopebc.org/weekly-bulletin,"7212 Keystone Street, Philadelphia, PA 19135",https://www.google.com/maps/?q=7212+Keystone+S...,['Anyone can access this program.'],...,-75.034054,19135.0,False,Closed,11:00 AM - 1:00 PM EST,Closed,Closed,Closed,Closed,Closed
117,Food Pantry,https://www.auntbertha.com//church-of-the-holy...,['food pantry'],NaN,"['all ages', 'i